In [1]:
import numpy as np
import pandas as pd

In [128]:
def kvm_fit(data, label, iters, lr, lambda_param):
    n_samples, n_features = data.shape
    y_ = np.where(label <= 0, -1, 1)
    w = np.zeros(n_features)
    b = 0
    for _ in range(iters):
        for idx, x_i in enumerate(data):
            condition = y_[idx] * (np.dot(x_i, w) - b) >= 1
            if condition:
                w -= lr * (2 * lambda_param * w)
            else:
                w -= lr * (2 * lambda_param * w - np.dot(x_i, y_[idx]))
                b -= lr * y_[idx]
    return w, b

In [242]:
trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

In [241]:
group = trainData.groupby('label')
zero = pd.DataFrame(group.get_group(0))
one = pd.DataFrame(group.get_group(1))
two = pd.DataFrame(group.get_group(2))
three = pd.DataFrame(group.get_group(3))
four = pd.DataFrame(group.get_group(4))
five = pd.DataFrame(group.get_group(5))
six = pd.DataFrame(group.get_group(6))
seven = pd.DataFrame(group.get_group(7))
eight = pd.DataFrame(group.get_group(8))
nine = pd.DataFrame(group.get_group(9))

In [53]:
zero_data = zero.drop(columns=['label'])
one_data = one.drop(columns=['label'])
two_data = two.drop(columns=['label'])
three_data = three.drop(columns=['label'])
four_data = four.drop(columns=['label'])
five_data = five.drop(columns=['label'])
six_data = six.drop(columns=['label'])
seven_data = seven.drop(columns=['label'])
eight_data = eight.drop(columns=['label'])
nine_data = nine.drop(columns=['label'])

In [74]:
rdm_zero = zero.sample(100)
rdm_one = one.sample(100)
rdm_two = two.sample(100)
rdm_three = three.sample(100)
rdm_four = four.sample(100)
rdm_five = five.sample(100)
rdm_six = six.sample(100)
rdm_seven = seven.sample(100)
rdm_eight = eight.sample(100)
rdm_nine = nine.sample(100)

In [240]:
sample_set = rdm_zero
sample_set = sample_set.append(rdm_one)
sample_set = sample_set.append(rdm_two)
sample_set = sample_set.append(rdm_three)
sample_set = sample_set.append(rdm_four)
sample_set = sample_set.append(rdm_five)
sample_set = sample_set.append(rdm_six)
sample_set = sample_set.append(rdm_seven)
sample_set = sample_set.append(rdm_eight)
sample_set = sample_set.append(rdm_nine)

In [90]:
def relabel(num, sample_set):
    binary = sample_set.copy(deep=True)
    binary.loc[binary['label'] != num, 'label'] = -1
    binary.loc[binary['label'] == num, 'label'] = 1
    return binary

In [170]:
class_zero = relabel(0, sample_set)
class_one = relabel(1, sample_set)
class_two = relabel(2, sample_set)
class_three = relabel(3, sample_set)
class_four = relabel(4, sample_set)
class_five = relabel(5, sample_set)
class_six = relabel(6, sample_set)
class_seven = relabel(7, sample_set)
class_eight = relabel(8, sample_set)
class_nine = relabel(9, sample_set)

In [179]:
def training(data):
    raw_set = data.copy(deep=True)
    raw_set_label = raw_set['label']
    raw_set_label = raw_set_label.values
    raw_set_data = raw_set.drop(columns=['label'], inplace=True)
    raw_set_data = raw_set.values/255.0
    [w,b] = kvm_fit(raw_set_data, raw_set_label, 1000, 0.001, 0.01)
    return w, b

In [200]:
w = [None] * 10
b = [None] * 10
[w[0], b[0]] = training(class_zero)
[w[1], b[1]] = training(class_one)
[w[2], b[2]] = training(class_two)
[w[3], b[3]] = training(class_three)
[w[4], b[4]] = training(class_four)
[w[5], b[5]] = training(class_five)
[w[6], b[6]] = training(class_six)
[w[7], b[7]] = training(class_seven)
[w[8], b[8]] = training(class_eight)
[w[9], b[9]] = training(class_nine)

In [181]:
def svm_predict(data, w, b):
    approx = np.dot(data, w) - b
    return approx

In [246]:
testCut = testData.copy(deep=True)
label = testCut['label']
testCut.drop(columns = ['label'], inplace=True)
testCut = testCut.values/255.0

correct = 0
for j in range(testCut.shape[0]):
    piece = testCut[j]
    result = [None] * 10
    for i in range(10):
        result[i] = svm_predict(piece, w[i], b[i])
    value = result.index(max(result))
    if (value == label[j]):
        correct = correct + 1

accuracy = correct / label.shape[0]
accuracy

0.79325